In [1]:
from preprocess import preprocess, pipeline
from utils import drop_useless, incomplete_columns, min_miss_value_corr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


data_test=pd.read_csv('../../data/test_airbnb_berlin.xls')
data_train=pd.read_csv('../../data/train_airbnb_berlin.xls')

On visualise le nombre de lignes, colonnes ainsi que les différentes features dans le jeu de données.

In [2]:

rows = len(data_train.axes[0])
print(f'Nombre de lignes: {rows}\n')

cols = len(data_train.axes[1])
print(f'Nombre de colonnes: {cols}\n')

print(f"Colonnes:{data_train.columns.values}")

Nombre de lignes: 15692

Nombre de colonnes: 39

Colonnes:['Listing ID' 'Listing Name' 'Host ID' 'Host Name' 'Host Since'
 'Host Response Time' 'Host Response Rate' 'Is Superhost' 'neighbourhood'
 'Neighborhood Group' 'City' 'Postal Code' 'Country Code' 'Country'
 'Latitude' 'Longitude' 'Is Exact Location' 'Property Type' 'Room Type'
 'Accomodates' 'Bathrooms' 'Bedrooms' 'Beds' 'Square Feet'
 'Guests Included' 'Min Nights' 'Reviews' 'First Review' 'Last Review'
 'Overall Rating' 'Accuracy Rating' 'Cleanliness Rating' 'Checkin Rating'
 'Communication Rating' 'Location Rating' 'Value Rating'
 'Instant Bookable' 'Business Travel Ready' 'Price']


In [3]:
data_train.head(1)

,Listing ID,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,19665213.0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0


On enlève les features qui ne vont pas apporter d'informations.

In [4]:
data_train=drop_useless(data_train)

On remarque la présence de quelques astérix dans le dataset: nous choisissons de les remplacer par des NAN.

In [5]:
data_train = data_train.replace("*", np.nan)

__Analyse du taux de missing par feature__

On liste les features pour lesquelles il manque des données.


In [6]:
listfeatures=incomplete_columns(data_train,to_print=True)



Square Feet => 15389 [98.07%]
Host Response Rate => 7249 [46.2%]
Host Response Time => 7075 [45.09%]
Checkin Rating => 2973 [18.95%]
Value Rating => 2972 [18.94%]
Accuracy Rating => 2971 [18.93%]
Cleanliness Rating => 2970 [18.93%]
Communication Rating => 2970 [18.93%]
Location Rating => 2971 [18.93%]
Overall Rating => 2962 [18.88%]
Min Nights => 249 [1.59%]
Property Type => 75 [0.48%]
Bathrooms => 25 [0.16%]
Beds => 20 [0.13%]
Accomodates => 19 [0.12%]
Bedrooms => 16 [0.1%]
Guests Included => 10 [0.06%]
Price => 9 [0.06%]


__Gestion des features à taux de "missing" extrême__

Lorsque l'on regarde les features où il manque moins de 30 données, on se rend compte que l'absence de ces données est MCAR. On se permet donc de supprimer les lignes où ces données sont manquantes. 
Il y a une exception sur la feature 'Is Superhost': si pour ces host il y a une note générale au dessus de 80, on les catégorises en Superhost, sinon, non. 

On commence par supprimer les features qui sont trop souvent manquantes (plus de 45%).

In [7]:
data_train=data_train.drop(columns=[x for x in listfeatures.keys() if listfeatures[x]['pct']>45])

Pour la colonne 'Property Type' on remarque que x% des propriétés sont des appartements, afin de simplifier le preprocessing, on considère que les valeurs manquantes sont des 'Apartment'

In [8]:
data_train['Property Type']=np.where(data_train['Property Type'].isna(), 'Apartment', data_train['Property Type'])

Si le prix manque, on enlève la ligne 

In [9]:
data_train=data_train.dropna(subset=['Price']).reset_index(drop=True)

On change les objects qui sont des objects.

In [10]:
obj=set(data_train.select_dtypes(["object"]).columns)
na=set(data_train.columns[data_train.isna().any()].tolist())
data_train=data_train.astype({x:"float64" for x in obj.intersection(na)})

In [11]:
data_train.head()

,Neighborhood Group,Latitude,Longitude,Property Type,Room Type,Accomodates,Bathrooms,Bedrooms,Beds,Guests Included,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,Pankow,52.54652,13.41792,Apartment,Private room,2.0,1.0,1.0,1.0,1.0,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0
1,Pankow,52.56512,13.42214,Apartment,Entire home/apt,2.0,1.0,2.0,2.0,2.0,...,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,f,41.0
2,Pankow,52.54741,13.42521,Apartment,Entire home/apt,3.0,1.0,1.0,2.0,1.0,...,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,50.0
3,Friedrichshain-Kreuzberg,52.50958,13.45144,Apartment,Private room,2.0,1.0,1.0,1.0,1.0,...,100.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f,50.0
4,Pankow,52.52995,13.41558,Apartment,Private room,3.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f,55.0


__Recherche de corrélations entre les features présentant un taux de missing value "NaN" autour de 18%__

On va s'aider de la liste **listfeatures** définie plus tôt


In [12]:
listfeatures_between_15_20 = [x for x in listfeatures.keys() if listfeatures[x]['pct']<20 and listfeatures[x]['pct']>15]

min_corr_pct=min_miss_value_corr(data_train,listfeatures_between_15_20)

print(f"Pourcentage minimal de corrélation entre deux features de la liste : {min_corr_pct}% \n")


Pourcentage minimal de corrélation entre deux features de la liste : 99.63% 



On peut donc raisonnablement affirmer que ces features sont de type MNAR : une listwise deletion n'est donc pas conseillée (et avec 18% d'instances concernées, ce n'est pas idéal)